In [1]:
import k3d
import numpy as np

import sys
sys.path.insert(0, "..")

from TimingCalib.MPMT import MPMT
from TimingCalib.PMT import PMT
from TimingCalib.LED import LED
from TimingCalib.WCD import WCD

from scipy.spatial.transform import Rotation as R

In [ ]:
# place mPMTs according to WCTE model - moved to main WCD code ... but kept here in case changes needed

wcte_height = 3450. # mm separation of mPMT opposite baseplates top-bottom
wcte_diameter = 3654. # mm separation of mPMT baseplates on opposite wall locations
wall_vertical_pitch = 580. # mm separation of mPMT centres for rows
tb_pitch = 580. # mm separation of mPMT centres on top and bottom (x and y the same)

loc_sig = [1.0, 1.0, 1.0] # mm positioning accuracy
rot_angles_sig = [0.001,0.001,0.001] # rad rotational angle positioning accuracy

# WCTE x-axis aligned with beam, z-axis vertical origin is centre
# three separate groups of mPMTs: bottom, wall, top

# Start with mPMT at centre of bottom, then order like the PMTs in an mPMT
# Next do rows starting at phi = 0

bottom_mpmts = []
#################

loc_centre = [0.,0.,-wcte_height/2.]
offsets = [[0.,0.,0.]]

offs = [-tb_pitch,0,tb_pitch]
for i in range(8):
    offset = [offs[[2,2,1,0,0,0,1,2][i]], offs[[1,2,2,2,1,0,0,0][i]],0.]
    offsets.append(offset)

offs = [-2.*tb_pitch,-tb_pitch,0,tb_pitch,2.*tb_pitch]
for i in range(12):
    offset = [offs[[4,4,3,2,1,0,0,0,1,2,3,4][i]], offs[[2,3,4,4,4,3,2,1,0,0,0,1][i]],0.] 
    offsets.append(offset)
                   
for offset in offsets:
    location = np.add(loc_centre,offset)               
    bottom_mpmts.append({
        'kind':'M2',
        'loc':location,
        'loc_sig':loc_sig,
        'rot_axes':'XYZ',
        'rot_angles':[0.,0.,0.],
        'rot_angles_sig':rot_angles_sig
})
    
wall_mpmts = []
###############

n_col = 18
for i_row in range(-2,3):
    loc = [wcte_diameter/2.,0.,i_row*wall_vertical_pitch]
    for j_col in range(n_col):
        phi_angle = 2.*np.pi*j_col/n_col
        rot_phi = R.from_euler('Z', phi_angle)
        rot_loc = rot_phi.apply(loc)
        # rotations of the normal defined by 2 extrinsic rotations
        rot_angles = [-np.pi/2.,phi_angle]
        wall_mpmts.append({'kind': 'M2',
                          'loc': rot_loc,
                          'loc_sig': loc_sig,
                          'rot_axes': 'yz',
                          'rot_angles': rot_angles,
                          'rot_angles_sig': [0.01, 0.01]})

        
top_mpmts = []
#################

loc_centre = [0.,0.,wcte_height/2.]

for offset in offsets:
    location = np.add(loc_centre,offset)               
    top_mpmts.append({
        'kind':'M2',
        'loc':location,
        'loc_sig':loc_sig,
        'rot_axes':'yz',
        'rot_angles':[np.pi,0.],
        'rot_angles_sig':[0.01, 0.01]
})


# define a new kind of WCD
wcd_kind = 'wcte'
WCD.prop_mean[wcd_kind] = WCD.def_prop_mean.copy()
WCD.prop_scale[wcd_kind] = WCD.def_prop_scale.copy()
WCD.prop_var[wcd_kind] = WCD.def_prop_var.copy()

WCD.mpmts_design[wcd_kind] = bottom_mpmts + wall_mpmts + top_mpmts

wcte = WCD(None, 'wcte', {}, {})


In [2]:
wcte = WCD(None, 'WCTE', {}, {})

In [3]:
# Show numbering and orientation of all mPMTs in the WCTE detector

plot = k3d.plot()

# mpmt coordinates
color_z = 0x0000ff
color_x = 0xff0000
origins = []
z_vecs = []
x_vecs = []
vec_length = 100. # length of x,z vectors to show for each pmt

color_mpmt = 0xabb2b9
n_point_mpmt = 8
indices_mpmt = []
for i in range(n_point_mpmt):
    indices_mpmt.append([i,(i+1)%n_point_mpmt])

# draw the extent of the mpmt baseplates
for i_mpmt,mpmt in enumerate(wcte.mpmts):
    
    location, direction_x, direction_z = mpmt.get_orientation('design')
    # lists to show mPMT coordinate systems
    z_vec = list(np.array(direction_z)*vec_length)
    x_vec = list(np.array(direction_x)*vec_length)
    origins.append(location)
    z_vecs.append(z_vec)
    x_vecs.append(x_vec)
    
    baseplate_points = np.array(mpmt.get_xy_points('design'),dtype=np.float32)
    plt_baseplate = k3d.lines(baseplate_points, indices_mpmt, indices_type='segment', color=color_mpmt)
    plot += plt_baseplate
    
    # k3d complains about the following not being float32!
    plt_text = k3d.text(str(i_mpmt), position=location, reference_point='cc', size=1., label_box=False, color=color_mpmt)
    plot += plt_text
    
plt_z_vecs = k3d.vectors(origins=origins, vectors=z_vecs, color=color_z, head_size=250.)
plot += plt_z_vecs
plt_x_vecs = k3d.vectors(origins=origins, vectors=x_vecs, color=color_x, head_size=250.)
plot += plt_x_vecs    

plot.display()

/gpfs/fs1/home/k/karlen/karlen/wcte/lib/python3.9/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

In [ ]:
# Show all mPMTs and some PMTs in the WCTE detector

plot = k3d.plot()

# mpmt coordinates
color_z = 0x0000ff
color_x = 0xff0000
origins = []
z_vecs = []
x_vecs = []
vec_length = 100. # length of x,z vectors to show for each pmt

color_mpmt = 0xabb2b9
n_point_mpmt = 8
indices_mpmt = []
for i in range(n_point_mpmt):
    indices_mpmt.append([i,(i+1)%n_point_mpmt])
    
# pmt faces:
color_pmt = 0xdc7633
n_point_pmt = 20
indices_pmt = []
for i in range(n_point_pmt):
    indices_pmt.append([i,(i+1)%n_point_pmt])

# draw the extent of the mpmt baseplates
for i_mpmt,mpmt in enumerate(wcte.mpmts):
    
    location, direction_x, direction_z = mpmt.get_orientation('design')
    # lists to show mPMT coordinate systems
    z_vec = list(np.array(direction_z)*vec_length)
    x_vec = list(np.array(direction_x)*vec_length)
    origins.append(location)
    z_vecs.append(z_vec)
    x_vecs.append(x_vec)
    
    baseplate_points = np.array(mpmt.get_xy_points('design'),dtype=np.float32)
    plt_baseplate = k3d.lines(baseplate_points, indices_mpmt, indices_type='segment', color=color_mpmt)
    plot += plt_baseplate
    
    if i_mpmt%10 == 0:
        for i_pmt, pmt in enumerate(mpmt.pmts):

            # k3d expects ndarray of float32 for segments
            circle_points = np.array(pmt.get_circle_points(n_point_pmt,'design'),dtype=np.float32)
            plt_circle = k3d.lines(circle_points, indices_pmt, indices_type='segment', color=color_pmt)
            plot += plt_circle
    
    # k3d complains about the following not being float32!
    #plt_text = k3d.text(str(i_mpmt), position=location, reference_point='cc', size=1., label_box=False, color=color_mpmt)
    #plot += plt_text
    
#plt_z_vecs = k3d.vectors(origins=origins, vectors=z_vecs, color=color_z, head_size=250.)
#plot += plt_z_vecs
#plt_x_vecs = k3d.vectors(origins=origins, vectors=x_vecs, color=color_x, head_size=250.)
#plot += plt_x_vecs    

plot.display()